In [8]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display
from sklearn import metrics 
import math
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

In [226]:
df2=pd.read_csv("energy_dataset.csv")
df1=pd.read_csv("weather_features.csv")

df=pd.concat([df1, df2], axis=1, join='inner')





### Bu veri analizinde sorumuz : Soğuk havalarda elektirik tüketimi ne kadardır?
#### Bunun için iki tane cvs dosyamızı ortak kolonlardan birleştirme işlemi yaptık (inner join ile)
##### Verilerimizi datetime'a göre sıralayacağız

In [227]:
df

,dt_iso,city_name,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,Valencia,270.475,270.475,270.475,1001,77,1,62,0.0,...,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,Valencia,270.475,270.475,270.475,1001,77,1,62,0.0,...,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,...,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,...,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,...,189.0,0.0,4935.0,9.0,NaN,4861.0,21785.0,20264.0,38.41,56.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2018-12-28 10:00:00+01:00,Valencia,282.140,281.150,283.150,1028,71,3,300,0.0,...,277.0,0.0,3113.0,96.0,NaN,3253.0,30619.0,30653.0,68.85,77.02
35060,2018-12-28 11:00:00+01:00,Valencia,282.150,282.150,282.150,1029,87,2,310,0.3,...,280.0,0.0,3288.0,51.0,NaN,3353.0,29932.0,29735.0,68.40,76.16
35061,2018-12-28 12:00:00+01:00,Valencia,284.150,284.150,284.150,1028,76,1,290,0.0,...,286.0,0.0,3503.0,36.0,NaN,3404.0,27903.0,28071.0,66.88,74.30
35062,2018-12-28 13:00:00+01:00,Valencia,285.660,285.150,286.150,1028,76,1,0,0.0,...,287.0,0.0,3586.0,29.0,NaN,3273.0,25450.0,25801.0,63.93,69.89


#### Verimizde Nan değerler olup olmadığını tespit ediyoruz.

In [228]:
df.isnull().sum()

dt_iso                                             0
city_name                                          0
temp                                               0
temp_min                                           0
temp_max                                           0
pressure                                           0
humidity                                           0
wind_speed                                         0
wind_deg                                           0
rain_1h                                            0
rain_3h                                            0
snow_3h                                            0
clouds_all                                         0
weather_id                                         0
weather_main                                       0
weather_description                                0
weather_icon                                       0
time                                               0
generation biomass                            

### Verimizde Nan veriler var bunlardan kurtulmamız gerek ayırca veri setimizde sadece bir tane string değişken var bunuda kategorik değişken olarak numeric ifadeye çevirmemiz gerekiyor
#### Ayrıca veri setimizi istersek Day/Month/Year olarak ayrı kolonlara ayıra biliriz
##### Öncelikle kategorik değişken dönüşümü yapalım


In [229]:
def train_cats(df):
    for n,c in df.items():
        if is_string_dtype(c):
            df[n]=c.astype("category").cat.as_ordered()

In [230]:
train_cats(df)

In [231]:
df["city_name"].cat.codes

0        0
1        0
2        0
3        0
4        0
        ..
35059    0
35060    0
35061    0
35062    0
35063    0
Length: 35064, dtype: int8

##### Tek bir string ifade olduğu için "Valencia" ifadesi numeric olarak 0 a döndü.

In [232]:
df["city_name"].cat.categories

Index(['Valencia'], dtype='object')

In [233]:
df3=df[["dt_iso","city_name","temp","temp_min","temp_max","humidity","price day ahead"]]

## VERİ SETİMİZİ BÖLDÜK İNCELEME YAPACAĞIM KOLUNLAR Sıcaklık değerleri,tarih ve günlük yapılan harcama bu işimizi bir nebze kolaylaştıracak

In [234]:
df3["city_name"].cat.categories

Index(['Valencia'], dtype='object')

In [235]:
df3["city_name"].cat.codes

0        0
1        0
2        0
3        0
4        0
        ..
35059    0
35060    0
35061    0
35062    0
35063    0
Length: 35064, dtype: int8

In [236]:
df=df3
df

,dt_iso,city_name,temp,temp_min,temp_max,humidity,price day ahead
0,2015-01-01 00:00:00+01:00,Valencia,270.475,270.475,270.475,77,50.10
1,2015-01-01 01:00:00+01:00,Valencia,270.475,270.475,270.475,77,48.10
2,2015-01-01 02:00:00+01:00,Valencia,269.686,269.686,269.686,78,47.33
3,2015-01-01 03:00:00+01:00,Valencia,269.686,269.686,269.686,78,42.27
4,2015-01-01 04:00:00+01:00,Valencia,269.686,269.686,269.686,78,38.41
...,...,...,...,...,...,...,...
35059,2018-12-28 10:00:00+01:00,Valencia,282.140,281.150,283.150,71,68.85
35060,2018-12-28 11:00:00+01:00,Valencia,282.150,282.150,282.150,87,68.40
35061,2018-12-28 12:00:00+01:00,Valencia,284.150,284.150,284.150,76,66.88
35062,2018-12-28 13:00:00+01:00,Valencia,285.660,285.150,286.150,76,63.93


### Nan Değerleri kaldırma ve train,valid veri setini oluşturuyoruz RMSLE ve RMSE ölçümü yapacağız

In [237]:
def fix_missing(df,col,name,nan_dict,is_train):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum():
            df[name+"_na"]=pd.isnull(col)
            nan_dict[name]=col.median()
            df[name]=col.fillna(nan_dict[name])
    else:
        if is_numeric_dtype(col):
            if name in nan_dict:
                df[name+"_na"]=pd.isnull(col)
                df[name]=col.fillna(nan_dict[name])
            else:
                df[name]=col.fillna(df[name].median())

In [238]:
def numericalize(df,col,name):
    if not is_numeric_dtype(col):
        df[name]=col.cat.codes+1

In [239]:
def split_train_val(df,n):
    return df[:n].copy(),df[n:].copy()

In [240]:
n_valid=17000
n_train=len(df)-n_valid
raw_train,raw_valid=split_train_val(df,n_train)

In [241]:
x_train,y_train,nas=proc_df(raw_train,'price day ahead')
x_valid,y_valid=proc_df(raw_valid,'price day ahead',nan_dict=nas,is_train=False)

In [242]:
x_valid

,dt_iso,city_name,temp,temp_min,temp_max,humidity
18064,18017,1,280.15,280.15,280.15,100
18065,18018,1,280.15,280.15,280.15,100
18066,18019,1,281.15,281.15,281.15,93
18067,18020,1,280.15,280.15,280.15,93
18068,18021,1,280.15,280.15,280.15,100
...,...,...,...,...,...,...
35059,34979,1,282.14,281.15,283.15,71
35060,34980,1,282.15,282.15,282.15,87
35061,34981,1,284.15,284.15,284.15,76
35062,34982,1,285.66,285.15,286.15,76


In [243]:
y_valid

array([62.52, 67.33, 85.38, ..., 66.88, 63.93, 64.27])

### RMSE incelemesini yapalım ilk önce harcama değerlerimizin ortlaması ve farklarını bulalım

In [244]:
df["price day ahead"].mean().sum()

49.87434120465434

In [245]:
df["price day ahead"].mean().sum()-df["price day ahead"]

0        -0.225659
1         1.774341
2         2.544341
3         7.604341
4        11.464341
           ...    
35059   -18.975659
35060   -18.525659
35061   -17.005659
35062   -14.055659
35063   -14.395659
Name: price day ahead, Length: 35064, dtype: float64

In [246]:
df=df.rename({'price day ahead' : 'price'}, axis=1)

### Kolon adını kolay olması için price olarak değiştirdim

In [247]:
math.sqrt(((df.price-df.price.mean()).pow(2).sum())/len(df))

14.618691223678443

#### Veri setimizdeki değerler Kelvin cinsinden yeni bir kolon oluşturup Celcius cinsinden değerlerini bulup filtreleme yapmak istiyorum

In [248]:
df["Temp_Cel"]=df["temp"].values-270

In [250]:
df["Temp_Cel"].mean().sum()

20.8003154009454

### Ortlama Sıcaklık değeri 20.8 C buna göre filtreleme işlemi yapacağım

In [272]:
filt=(df.Temp_Cel>21)
df1=df[filt]
df2=df[~filt]

### Veri setimi ortalama 25 Celcius üstü ve altı olarak iki ye böldüm 

In [273]:
math.sqrt(((df.price-df.price.mean()).pow(2).sum())/len(df))

14.618691223678443

In [274]:
math.sqrt(((df1.price-df1.price.mean()).pow(2).sum()+(df2.price-df2.price.mean()).pow(2).sum())/len(df))

14.608008244332323

In [281]:
filt1=(df2.Temp_Cel<15)
df3=df2[filt1]
df4=df2[~filt1]

In [282]:
df3

,dt_iso,city_name,temp,temp_min,temp_max,humidity,price,Temp_Cel
0,2015-01-01 00:00:00+01:00,Valencia,270.475,270.475,270.475,77,50.10,0.475
1,2015-01-01 01:00:00+01:00,Valencia,270.475,270.475,270.475,77,48.10,0.475
2,2015-01-01 02:00:00+01:00,Valencia,269.686,269.686,269.686,78,47.33,-0.314
3,2015-01-01 03:00:00+01:00,Valencia,269.686,269.686,269.686,78,42.27,-0.314
4,2015-01-01 04:00:00+01:00,Valencia,269.686,269.686,269.686,78,38.41,-0.314
...,...,...,...,...,...,...,...,...
35057,2018-12-28 08:00:00+01:00,Valencia,282.140,281.150,283.150,81,68.01,12.140
35058,2018-12-28 09:00:00+01:00,Valencia,281.630,280.150,283.150,76,68.58,11.630
35059,2018-12-28 10:00:00+01:00,Valencia,282.140,281.150,283.150,71,68.85,12.140
35060,2018-12-28 11:00:00+01:00,Valencia,282.150,282.150,282.150,87,68.40,12.150


In [283]:
df4

,dt_iso,city_name,temp,temp_min,temp_max,humidity,price,Temp_Cel
14,2015-01-01 14:00:00+01:00,Valencia,285.050656,285.050656,285.050656,52,47.35,15.050656
15,2015-01-01 15:00:00+01:00,Valencia,285.277344,285.277344,285.277344,50,43.61,15.277344
35,2015-01-02 11:00:00+01:00,Valencia,285.595000,285.595000,285.595000,52,10.00,15.595000
36,2015-01-02 12:00:00+01:00,Valencia,286.304000,286.304000,286.304000,44,7.04,16.304000
37,2015-01-02 13:00:00+01:00,Valencia,285.595000,285.595000,285.595000,52,7.00,15.595000
...,...,...,...,...,...,...,...,...
35041,2018-12-27 16:00:00+01:00,Valencia,289.150000,289.150000,289.150000,63,54.87,19.150000
35042,2018-12-27 17:00:00+01:00,Valencia,288.150000,288.150000,288.150000,62,52.60,18.150000
35043,2018-12-27 18:00:00+01:00,Valencia,286.640000,286.150000,287.150000,71,51.09,16.640000
35062,2018-12-28 13:00:00+01:00,Valencia,285.660000,285.150000,286.150000,76,63.93,15.660000


In [284]:
math.sqrt(((df1.price-df1.price.mean()).pow(2).sum()+(df3.price-df3.price.mean()).pow(2).sum()+(df4.price-df4.price.mean()).pow(2).sum())/len(df))

14.575648623027094

In [289]:
filt2=(df1.Temp_Cel<27)
df5=df1[filt2]
df6=df1[~filt2]

In [290]:
df5

,dt_iso,city_name,temp,temp_min,temp_max,humidity,price,Temp_Cel
230,2015-01-10 13:00:00+01:00,Valencia,292.400,292.400,292.400,28,56.80,22.400
231,2015-01-10 14:00:00+01:00,Valencia,293.950,293.950,293.950,26,53.60,23.950
232,2015-01-10 15:00:00+01:00,Valencia,292.681,292.681,292.681,38,53.50,22.681
233,2015-01-10 16:00:00+01:00,Valencia,294.850,294.850,294.850,25,54.90,24.850
234,2015-01-10 17:00:00+01:00,Valencia,291.231,291.231,291.231,43,55.26,21.231
...,...,...,...,...,...,...,...,...
34945,2018-12-23 16:00:00+01:00,Valencia,292.660,292.150,293.150,59,67.01,22.660
34946,2018-12-23 17:00:00+01:00,Valencia,291.660,291.150,292.150,55,63.75,21.660
34968,2018-12-24 15:00:00+01:00,Valencia,291.160,290.150,292.150,51,67.34,21.160
34969,2018-12-24 16:00:00+01:00,Valencia,291.660,291.150,292.150,45,64.64,21.660


In [291]:
df6

,dt_iso,city_name,temp,temp_min,temp_max,humidity,price,Temp_Cel
2079,2015-03-28 14:00:00+01:00,Valencia,297.650344,297.650344,297.650344,24,37.69,27.650344
2080,2015-03-28 15:00:00+01:00,Valencia,298.067656,298.067656,298.067656,24,35.50,28.067656
2082,2015-03-28 17:00:00+01:00,Valencia,297.306344,297.306344,297.306344,24,40.00,27.306344
2103,2015-03-29 15:00:00+02:00,Valencia,297.940000,297.940000,297.940000,26,10.55,27.940000
2104,2015-03-29 16:00:00+02:00,Valencia,298.379000,298.379000,298.379000,25,12.40,28.379000
...,...,...,...,...,...,...,...,...
33518,2018-10-25 16:00:00+02:00,Valencia,300.150000,300.150000,300.150000,28,49.68,30.150000
33519,2018-10-25 17:00:00+02:00,Valencia,298.150000,298.150000,298.150000,44,46.99,28.150000
33927,2018-11-11 15:00:00+01:00,Valencia,297.150000,297.150000,297.150000,47,67.89,27.150000
33928,2018-11-11 16:00:00+01:00,Valencia,297.150000,297.150000,297.150000,47,68.25,27.150000


In [292]:
math.sqrt(((df5.price-df5.price.mean()).pow(2).sum()+(df6.price-df6.price.mean()).pow(2).sum()+(df3.price-df3.price.mean()).pow(2).sum()+(df4.price-df4.price.mean()).pow(2).sum())/len(df))

14.555235671034568

### RMSLE olarak yaklaşalım ve R^2 olarak yaklaşalım

In [307]:
m=RandomForestRegressor(n_estimators=1,bootstrap=False,n_jobs=-1)
m.fit(x_train,y_train)
m.score(x_train,y_train)

0.9998752722624353

In [308]:
def rmse(x,y):
    return math.sqrt(((x-y)**2).mean())

In [309]:
def print_score(m):
    print(f"RMSE of valid set {rmse(m.predict(x_valid),y_valid)}")
    print(f"R^2 of valid set {m.score(x_valid,y_valid)}")
    print(f"RMSE of train set {rmse(m.predict(x_train),y_train)}")
    print(f"R^2 of train set {m.score(x_train,y_train)}")

In [310]:
print_score(m)

RMSE of valid set 15.69172529036356
R^2 of valid set -0.5526395574017449
RMSE of train set 0.16751673898090122
R^2 of train set 0.9998752722624353


In [311]:
m=RandomForestRegressor(n_estimators=10,bootstrap=False,n_jobs=-1)
%time m.fit(x_train,y_train)
print_score(m)

Wall time: 142 ms
RMSE of valid set 15.69172529036356
R^2 of valid set -0.5526395574017449
RMSE of train set 0.16751673898090122
R^2 of train set 0.9998752722624353


In [312]:
m=RandomForestRegressor(n_estimators=100,bootstrap=False,n_jobs=-1)
%time m.fit(x_train,y_train)
print_score(m)

Wall time: 1.29 s
RMSE of valid set 15.691725290363493
R^2 of valid set -0.5526395574017313
RMSE of train set 0.16751673898090122
R^2 of train set 0.9998752722624353


In [315]:
m=RandomForestRegressor(n_estimators=1000,bootstrap=False,n_jobs=-1)
%time m.fit(x_train,y_train)
print_score(m)

Wall time: 12.2 s
RMSE of valid set 15.69172529036287
R^2 of valid set -0.5526395574016081
RMSE of train set 0.16751673898090122
R^2 of train set 0.9998752722624353
